### 2 Обработка текста

В предыдущей главе мы углубились в структуру больших языковых моделей (LLM) и узнали, что они предварительно обучаются на огромных объемах текста, обрабатывая
по одному слову (токену) за раз. 

![](images/llm2.1.png)


### 2.1 Векторное представление (vector embedding)

Нейросетевые модели, не могут напрямую обрабатывать текст. Текст, как тип данных, несовместим с тензорными операциями. Следовательно, нужен способ представлять слова в виде векторов с непрерывными значениями. (ссылка узнать больше в Приложении A, раздел A2.2 «Понимание тензоров».)

Построение векторного представления может быть проведено разными способами (различными моделями). 
Прочие типы данных также требуют преобразования в тензорный формат.  

![](images/llm2.2.png)

Существует несколько алгоритмов для создания векторного представления текста. Важным моментом является выбор - что считать единичным токеном. Одним из первых и наиболее популярных примеров является Word2Vec (токен - слово). Основная идея заключается в том, что слова, встречающиеся в схожем контексте, имеют схожее значение. Следовательно, при проецировании в двумерное пространство, можно увидеть, что схожие термины группируются вместе.

![](images/llm2.3.png)

Помимо моделей токенизации слов, существуют модели токенизации предложений, абзацев, целых документов.
Таким образом, картинка из предыдущей главы (predicting next world) не вполне корректна - предсказывается следующий токен.

Вендоры LLM обычно создают свои собственные модели токенизации, которые являются частью входного слоя и обновляются во время обучения. Преимущество такого подхода - оптимизация для конкретной задачи и имеющихся данных. 

Размерность представления (двумерное на рисунке выше) может быть произвольной. Большая размерность может отражать более тонкие отношения, но за счет вычислительной эффективности. Эта размерность называется размерностью скрытых состояний модели (dimensionality of the model's hidden states). 

Эта размерность - компромисс между производительностью и эффективностью:  
- GPT-2 (125M параметров) использует размерность 768  
- GPT-3 (175B параметров) использует размерность 12 288  

### 2.2 Токенизация текста

![](images/llm2.4.png)

Будем токенизировать рассказ Эдит Уортон [«Вердикт»](https://en.wikisource.org/wiki/The_Verdict)

In [1]:
with open("source/the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()
print("Total number of character:", len(raw_text))
print(raw_text[:99])

Total number of character: 20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


Для разделения текста на токены, используем сначала команду re.split, чтобы разделить текст на пробельные символы:

In [2]:
import re
text = "Hello, world. This, is a test."
result = re.split(r'(\s)', text)
print(result)

['Hello,', ' ', 'world.', ' ', 'This,', ' ', 'is', ' ', 'a', ' ', 'test.']


Используем регулярные выражения для выделения запятых и точек:

In [3]:
result = re.split(r'([,.]|\s)', text)
print(result)

['Hello', ',', '', ' ', 'world', '.', '', ' ', 'This', ',', '', ' ', 'is', ' ', 'a', ' ', 'test', '.', '']


При разработке токенизатора, выделение пробельных символов зависит от приложения и его требований. Удаление пробелов снижает требования к памяти и вычислительным ресурсам. Однако сохранение пробелов может быть полезно, если мы обучаем модели, чувствительные к точной структуре текста (например, код Python, чувствительный к отступам и интервалам). 

Изменим код, чтобы он мог обрабатывать и другие типы пунктуации, такие как вопросительные знаки, кавычки и двойные тире, которые мы видели ранее в первых 100 символах рассказа Эдит Уортон, а также дополнительные специальные символы.

In [4]:
text = "Hello, world. Is this-- a test?"
result = re.split(r'([,.:;?_!"()\']|--|\s)', text)
result = [item.strip() for item in result if item.strip()]
print(result)

['Hello', ',', 'world', '.', 'Is', 'this', '--', 'a', 'test', '?']


Наша модель токенизации теперь может успешно обрабатывать различные специальные символы в тексте.

![](images/llm2.5.png)

Теперь, когда у нас есть работающий базовый токенизатор, применим его ко всему рассказу Эдит Уортон:

In [5]:
preprocessed = re.split(r'([,.?_!"()\']|--|\s)', raw_text)
preprocessed = [item.strip() for item in preprocessed if item.strip()]
print(len(preprocessed))
print(preprocessed[:30])

4649
['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that', ',', 'in']


### 2.3 Преобразование токенов в идентификаторы

Полученные токены сортируются в алфавитном порядке, повторяющиеся удаляются. Затем уникальные токены объединяются в словарь: текстовый токен - целочисленное значение.

![](images/llm2.6.png)

In [6]:
all_words = sorted(list(set(preprocessed)))
vocab_size = len(all_words)
print(vocab_size)

vocab = {token:integer for integer,token in enumerate(all_words)}
for i, item in enumerate(vocab.items()):
    print(item)
    if i > 15:
        break

1159
('!', 0)
('"', 1)
("'", 2)
('(', 3)
(')', 4)
(',', 5)
('--', 6)
('.', 7)
(':', 8)
(';', 9)
('?', 10)
('A', 11)
('Ah', 12)
('Among', 13)
('And', 14)
('Are', 15)
('Arrt', 16)


Выходные данные LLM будут поступать как последовательность числовых значений, которые поребуется преобразовать обратно в текст. Для этого мы можем создать обратную версию словаря (детокенизатор).

![](images/llm2.7.png)

Реализуем полный класс токенизатора и детокенизатора и применим этот словарь для преобразования всего текста в токены.

In [7]:
class SimpleTokenizerV1:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i:s for s,i in vocab.items()}
    
    def encode(self, text):
        preprocessed = re.split(r'([,.?_!"()\']|--|\s)', text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids
        
    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids]) 
        
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        return text
    
tokenizer = SimpleTokenizerV1(vocab)
 
text = """"It's the last he painted, you know," Mrs. Gisburn said with pardonable pride."""
ids = tokenizer.encode(text)

print(ids)
print(tokenizer.decode(ids))

[1, 58, 2, 872, 1013, 615, 541, 763, 5, 1155, 608, 5, 1, 69, 7, 39, 873, 1136, 773, 812, 7]
" It' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.



### 2.4 Добавление специальных токенов контекста

Изменим токенизатор для обработки неизвестных слов. Также обсудим использование и добавление специальных токенов контекста, которые могут улучшить понимание моделью контекста или другой соответствующей информации в тексте. Эти специальные токены могут включать, например, маркеры для неизвестных слов и границ документа.

В частности, мы изменим словарь и токенизатор для поддержки двух новых токенов: <|unk|> и <|endoftext|>

![](images/llm2.8.png)

Как показано на рисунке, мы можем изменить токенизатор, чтобы он использовал токен <|unk|>, если он встретит слово, которое не является частью словаря. Кроме того, мы добавляем токен между несвязанными текстами. Например, при обучении GPT-подобных LLM на нескольких независимых документах или книгах обычно вставляется токен перед каждым документом или книгой, следующим за предыдущим источником текста. Это помогает LLM понять, что, хотя эти текстовые источники объединены для обучения, на самом деле они не связаны друг с другом.

![](images/llm2.9.png)

In [8]:
all_tokens = sorted(list(set(preprocessed)))
all_tokens.extend(["<|endoftext|>", "<|unk|>"])
vocab = {token:integer for integer,token in enumerate(all_tokens)}
 
print(len(vocab.items()))

for i, item in enumerate(list(vocab.items())[-5:]):
    print(item)

class SimpleTokenizerV2:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = { i:s for s,i in vocab.items()}
    
    def encode(self, text):
        preprocessed = re.split(r'([,.?_!"()\']|--|\s)', text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        preprocessed = [item if item in self.str_to_int
                        else "<|unk|>" for item in preprocessed]
 
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids
        
    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
 
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        return text
    
text1 = "Hello, do you like tea?"
text2 = "In the sunlit terraces of the palace."
text = " <|endoftext|> ".join((text1, text2))
print(text)

tokenizer = SimpleTokenizerV2(vocab)
print(tokenizer.encode(text))
print(tokenizer.decode(tokenizer.encode(text)))

1161
('younger', 1156)
('your', 1157)
('yourself', 1158)
('<|endoftext|>', 1159)
('<|unk|>', 1160)
Hello, do you like tea? <|endoftext|> In the sunlit terraces of the palace.
[1160, 5, 362, 1155, 642, 1000, 10, 1159, 57, 1013, 981, 1009, 738, 1013, 1160, 7]
<|unk|>, do you like tea? <|endoftext|> In the sunlit terraces of the <|unk|>.


Выше мы видим, что идентификатор 1159 соответствует токену-разделителю <|endoftext|>, а идентификатор 1160 используются для неизвестных слов.

Сравнивая приведенный выше детокенизированный текст с исходным входным текстом, мы знаем, что набор обучающих данных не содержал слов «Привет» и «дворец».

Также рассматривают дополнительные специальные токены, такие как:
- [BOS] (начало последовательности): этот токен отмечает начало текста.   
- [EOS] (конец последовательности): этот токен располагается в конце текста. Например, при объединении двух разных статей или книг Википедии токен [EOS] указывает, где заканчивается одна статья и начинается следующая.  
- [PAD] (заполнение): при обучении LLM с размерами пакетов больше единицы пакет может содержать тексты различной длины. Чтобы гарантировать одинаковую длину всех текстов, более короткие тексты расширяются или «дополняются» с помощью токена [PAD] до длины самого длинного текста в пакете.  

### 2.5 Кодирование по парам байтов (Byte pair encoding)

В этом разделе рассматривается более сложная схема токенизации. Она использовалась для обучения GPT-2, GPT-3.

Поскольку реализация BPE может быть сложной, будем использовать существующую библиотеку Python с открытым исходным кодом под названием [tiktoken](https://github.com/openai/tiktoken), которая очень эффективно реализует алгоритм BPE на основе исходного кода на Rust. 

In [9]:
from importlib.metadata import version
import tiktoken
print("tiktoken version:", version("tiktoken"))

tokenizer = tiktoken.get_encoding("gpt2")
text = "Hello, do you like tea? <|endoftext|> In the sunlit terraces of someunknownPlace."
integers = tokenizer.encode(text, allowed_special={"<|endoftext|>"})
print(integers)

tiktoken version: 0.6.0
[15496, 11, 466, 345, 588, 8887, 30, 220, 50256, 554, 262, 4252, 18250, 8812, 2114, 286, 617, 34680, 27271, 13]


Обратим внимание:
- Во-первых, токену <|endoftext|> присваивается относительно большой идентификатор токена, а именно 50256. Фактически, токенизатор BPE, который использовался для обучения ChatGPT имеет общий размер словаря 50 257, причем <|endoftext|> присвоен самый большой идентификатор токена.

- Во-вторых, BPE правильно кодирует и декодирует неизвестные слова, такие как «someunknownPlace». Токенизатор BPE может обрабатывать любое неизвестное слово.

Алгоритм, лежащий в основе BPE, разбивает слова, которых нет в его предопределенном словаре, на более мелкие подслова или даже отдельные символы, что позволяет ему обрабатывать слова, которых нет в словаре. Таким образом, благодаря алгоритму BPE, если токенизатор встречает незнакомое слово во время токенизации, он может представить его как последовательность токенов или символов подслова, как показано на рисунке.

![](images/llm2.10.png)

Подробное обсуждение и реализация BPE выходит за рамки этой книги, но, если коротко, словарь строится путем итеративного объединения часто встречающихся символов в подслова, а часто встречающихся подслов — в слова. Например, BPE начинает с добавления в свой словарь всех отдельных одиночных символов («a», «b», ...). На следующем этапе он объединяет комбинации символов, которые часто встречаются вместе, в подслова. Например, «d» и «e» могут быть объединены в подслово «de», которое часто встречается во многих английских словах, таких как «define», «dependent», «made» и «hidden».

### 2.6 Выборка данных с помощью скользящего окна

Следующим шагом является создание пар вход - цель (input - target), необходимых для обучения LLM.
Реализуем теперь загрузчик данных, который извлекает пары из набора обучающих данных, используя подход скользящего окна.
Для чего токенизируем весь рассказ The Verdict, с помощью токенизатора BPE.

In [10]:
with open("source/the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()
 
enc_text = tokenizer.encode(raw_text)
print(len(enc_text))

5145


Интуитивно понятный способ создания пар данных — создание двух переменных, x и y, где x содержит входные токены, а y содержит целевые значения, которые представляют собой входные данные, сдвинутые на 1 (размер скользящего окна):

In [11]:
enc_sample = enc_text[50:]
context_size = 4

x = enc_sample[:context_size]
y = enc_sample[1:context_size+1]
print(f"x: {x}")
print(f"y:      {y}")

x: [290, 4920, 2241, 287]
y:      [4920, 2241, 287, 257]


Можно представить задачу предсказания следующего слова:

In [12]:
for i in range(1, context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]
    print(context, "---->", desired)

[290] ----> 4920
[290, 4920] ----> 2241
[290, 4920, 2241] ----> 287
[290, 4920, 2241, 287] ----> 257


Все, что находится слева от стрелки (---->), относится к входным данным, которые LLM получит, а идентификатор токена в правой части стрелки представляет собой целевой идентификатор токена, который LLM должен предсказать.

В целях иллюстрации давайте повторим предыдущий код, но преобразуем идентификаторы токенов в текст.

In [13]:
for i in range(1, context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]
    print(tokenizer.decode(context), "---->", tokenizer.decode([desired]))

 and ---->  established
 and established ---->  himself
 and established himself ---->  in
 and established himself in ---->  a


![](images/llm2.11.png)

Следующая задача - реализация эффективного загрузчика данных, который выполняет итерацию по входному набору данных и возвращает входные и целевые значения в виде тензоров PyTorch, каждая строка которых - один вход или одна цель (на рисунке слова, вместо идентификаторов). 

![](images/llm2.12.png)

Бдем использовать встроенные классы PyTorch Dataset и DataLoader. (ссылка Дополнительную информацию и рекомендации по установке PyTorch см. в разделе A.1.3 «Установка PyTorch» в Приложении A.)

In [14]:
import torch
from torch.utils.data import Dataset, DataLoader
 
class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.tokenizer = tokenizer
        self.input_ids = []
        self.target_ids = []
 
        token_ids = tokenizer.encode(txt)
 
        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i:i + max_length]
            target_chunk = token_ids[i + 1: i + max_length + 1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))
 
    def __len__(self):
        return len(self.input_ids)
 
    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]

Класс GPTDatasetV1 определяет, как отдельные строки извлекаются из набора данных (каждая строка состоит из нескольких идентификаторов токенов). 

(ссылка прочтите раздел A.6 «Настройка эффективных загрузчиков данных» в приложении A, в котором объясняется общая структура и использование классов PyTorch Dataset и DataLoader )

Определим далее загрузку тензоров с помощью PyTorch DataLoader:

In [15]:
def create_dataloader_v1(txt, batch_size=4, 
        max_length=256, stride=128, shuffle=True, drop_last=True):
    tokenizer = tiktoken.get_encoding("gpt2")
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)
    dataloader = DataLoader(
        dataset, batch_size=batch_size, shuffle=shuffle, drop_last=drop_last)
    return dataloader

In [16]:
with open("source/the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()
 
dataloader = create_dataloader_v1(
    raw_text, batch_size=1, max_length=4, stride=1, shuffle=False)
data_iter = iter(dataloader)
first_batch = next(data_iter)
print(first_batch[0])
print('    ', first_batch[1])

tensor([[  40,  367, 2885, 1464]])
     tensor([[ 367, 2885, 1464, 1807]])


Переменная first_batch содержит два тензора: первый тензор хранит идентификаторы входных токенов, а второй тензор хранит идентификаторы целевых токенов. Поскольку max_length установлено равным 4, каждый из двух тензоров содержит 4 идентификатора токена. Обратите внимание, что входной размер 4 относительно мал и выбран только для иллюстрации. Обычно LLM обучают с размером входных данных не менее 256.

Чтобы проиллюстрировать значение этого шага, возьмем еще один блок (batch) из этого набора данных (hазмеры блоков равные 1, cлужат для иллюстративных целей. Размер пакета — это компромисс и гиперпараметр, с которым можно экспериментировать при обучении LLM. Небольшие размеры пакетов требуют меньше памяти во время обучения, но приводят к более "шумным" обновлениям модели):

In [17]:
second_batch = next(data_iter)
print(second_batch[0])
print('    ', second_batch[1])

tensor([[ 367, 2885, 1464, 1807]])
     tensor([[2885, 1464, 1807, 3619]])


![](images/llm2.13.png)

Пример загрузчика с другими параметрами - увеличиваем шаг до 4. Это необходимо для полного использования данных (не пропускаем ни одного слова), и для избежания перекрытия между блоками (перекрытие может привести к переобучению).

In [18]:
dataloader = create_dataloader_v1(raw_text, batch_size=8, max_length=4, stride=4)
 
data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print("Inputs:\n", inputs)
print("\nTargets:\n", targets)

Inputs:
 tensor([[  198,   198,  1544,  6204],
        [   13,  8759,  2763,   438],
        [  503,   465,  7721,   257],
        [  286,   511,  6799,   454],
        [  314,  2993,  1576,   284],
        [  355,  1752,   530,   550],
        [  340,   656,  5563,   286],
        [  262, 23755,   286,   262]])

Targets:
 tensor([[  198,  1544,  6204,  2045],
        [ 8759,  2763,   438,  1169],
        [  465,  7721,   257,  1310],
        [  511,  6799,   454,    30],
        [ 2993,  1576,   284,  2666],
        [ 1752,   530,   550,   890],
        [  656,  5563,   286,  1242],
        [23755,   286,   262, 14005]])


### 2.7 Создание векторного представления из токенов

Последним шагом подготовки входного текста для обучения LLM является преобразование идентификаторов токенов в векторы - представления, как показано на рисунке. Векторное представление содержит информацию о связях токена с другими. Многообразие этих связей зависит от размерности представления (dimensionality hidden state). Сила этих связей изменяется в процессе обучения (изначально эти связи - веса матриц инициализированы случайными значениями). В части вычислений за изменения отвечает алгоритм обратного распространения ошибки.  (ссылка Если вы не знакомы с тем, как нейронные сети обучаются с помощью обратного распространения ошибки, прочтите раздел A.4 «Легкая автоматическая дифференциация» в Приложении A.)

![](images/llm2.14.png)

Проиллюстрируем, как работает векторное преобразование идентификатора токена на практическом примере. Предположим, у нас есть следующие четыре входных токена с идентификаторами 2, 3, 5 и 1. Для простоты предположим, что у нас есть небольшой словарь, состоящий всего из 6 слов (вместо 50 257 слов в словаре токенизатора BPE), и мы хотим создать представления размерности 3.

In [19]:
input_ids = torch.tensor([2, 3, 5, 1])
vocab_size = 6
output_dim = 3

In [20]:
torch.manual_seed(123) 
embedding_layer = torch.nn.Embedding(vocab_size, output_dim)
print(embedding_layer.weight)

Parameter containing:
tensor([[ 0.3374, -0.1778, -0.1690],
        [ 0.9178,  1.5810,  1.3010],
        [ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-1.1589,  0.3255, -0.6315],
        [-2.8400, -0.7849, -1.4096]], requires_grad=True)


Матрица имеет шесть (размер словаря) строк и три столбца (размерность представления, dimensionality hidden state). Значения в матрице (веса) инициализированы случайными числами и изменяются в процессе обучения. Значения в матрице как точки в трехмерном пространстве соответствуют связанности токенов: контекстно-близкие имеют меньшую метрику расстояния, контекстно-далекие - большую (более наглядно расстояние между токенами иллюстрируется на картинке для размерности 2 в начале главы). Один токен - одна строка.

In [21]:
print(embedding_layer(input_ids))
# выведены строки - токены с индексами input_ids = [2, 3, 5, 1]

tensor([[ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-2.8400, -0.7849, -1.4096],
        [ 0.9178,  1.5810,  1.3010]], grad_fn=<EmbeddingBackward0>)


![](images/llm2.16.png)

### 2.8 Кодирование позиций слов / векторно - позиционное представление (positional embedding)

Оставшимся неудобством в полученном векторном представлении токенов - независимость токена от положения во входящей последовательности. Т.е. один и тот же идентификатор токена всегда отображается в одно и то же векторное представление, независимо от того, где идентификатор токена расположен во входной последовательности (как на предыдущем рисунке).

В принципе, детерминированное, независимое от позиции представление полезно для целей воспроизводимости. Однако, различное представление в зависимости от положения в последовательности будет полезно для обучения, т.к. кодировать контекст. 

Добавить информацию о положении токена можно абсолютным способом: для каждой позиции токена в последовательности добавляется её код - к токену прибавляется значение, рисунок. 

![](images/llm2.17.png)

Или относительным способом, когда существенно относительное расстояние между токенами в последовательности (токен в начале предложения и в конце). В таком случае модель может лучше обобщать последовательности различной длины, даже если во время обучения она не встречала такой входящей длины.

Оба типа позиционных представлений направлены на расширение возможностей LLM понимать порядок и взаимосвязи между токенами, обеспечивая более точные и контекстно-зависимые результаты. Выбор между ними часто зависит от конкретного приложения и характера обрабатываемых данных.

Рассмотрим векторно - позиционное представление для относительно большой размерности 256 и словарь токенизатора BPE:

In [22]:
output_dim = 256
vocab_size = 50257
token_embedding_layer = torch.nn.Embedding(vocab_size, output_dim)

In [23]:
max_length = 4
dataloader = create_dataloader_v1(
    raw_text, batch_size=8, max_length=max_length, stride=max_length, shuffle=False)
data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print("Token IDs:\n", inputs)
print("\nInputs shape:\n", inputs.shape)

# блок (батч) данных состоит из 8 кусков текста по 4 токена в каждом.

Token IDs:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

Inputs shape:
 torch.Size([8, 4])


In [24]:
token_embeddings = token_embedding_layer(inputs)
print(token_embeddings.shape)

# пропустив вход через токенизатор, получим 256 слоев размерности. 

torch.Size([8, 4, 256])


Как мы можем сказать на основе выходных данных тензора размером 8x4x256, каждый идентификатор токена теперь внедрен как 256-мерный вектор.

Для подхода абсолютного внедрения модели GPT нам просто нужно создать еще один слой внедрения, который имеет то же измерение, что и token_embedding_layer:

In [25]:
context_length = max_length
pos_embedding_layer = torch.nn.Embedding(context_length, output_dim)
pos_embeddings = pos_embedding_layer(torch.arange(context_length))
print(pos_embeddings.shape)

# добавляем позиционный токенизатор, которому нужно закодировать 4 позиции во входящей строке

torch.Size([4, 256])


Входными данными для pos_embeddings обычно является вектор-заполнитель torch.arange(context_length), который содержит последовательность чисел 0, 1,..., вплоть до максимальной входной длины (4 в примере). На практике входной текст может быть длиннее, чем поддерживаемая длина контекста, и в этом случае нам придется обрезать текст.

In [26]:
input_embeddings = token_embeddings + pos_embeddings
print(input_embeddings.shape)

# итоговое векторное представление текста с учетом позиции, подходящее для обучения LLM. 

torch.Size([8, 4, 256])


Созданные нами input_embeddings, как показано на рисунке представляют собой примеры входных данных, которые теперь могут обрабатываться основными модулями LLM.

![](images/llm2.18.png)

2.9 Резюме

- LLM требуют преобразования текстовых данных в числовые векторы, известные как векторные представления, поскольку они не могут обрабатывать "сырой" текст. Представления преобразуют дискретные данные (например, слова или изображения) в непрерывные векторные пространства, делая их совместимыми с тензорными операциями.
- На первом этапе необработанный текст разбивается на токены, которые могут быть словами или символами. Затем токены преобразуются в целочисленные представления, называемые идентификаторами. Специальные токены, такие как <|unk|> и <|endoftext|>, можно добавлять для улучшения понимания модели и обработки различных контекстов, например неизвестных слов или обозначения границы между несвязанными текстами.
- Токенизатор кодирования по парам байтов (BPE) может эффективно обрабатывать неизвестные слова, разбивая их на подслова или отдельные символы.
- Мы используем подход скользящего окна для токенизированных данных для генерации пар входных-целевых данных для обучения LLM.
- Представление создается стандартными функциями, с учетом длины блока входящих даннных и размерностью скрытых состояний модели. 
- Представление токена можно улучшить, добавив представление позиции во входящем блоке. 